In [3]:
# import libraries

from bs4 import BeautifulSoup
import requests
import smtplib
import time
import datetime
import csv

In [5]:
# connect to Barnes and Noble

URL = 'https://www.barnesandnoble.com/w/frank-herberts-dune-saga-3-book-boxed-set-frank-herbert/1136810577?ean=9780593201893'

headers = {"from my computer"}

page = requests.get(URL, headers=headers) # connecting my computer to the URL

# bring in the data

soup1 = BeautifulSoup(page.content, "html.parser")

# prettify (formats in a nice way)
soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

In [3]:
# finding the title of the book
title = soup2.find_all(itemprop="name")[-1].get_text().strip()
print(title)

Frank Herbert's Dune Saga 3-Book Boxed Set: Dune, Dune Messiah, and Children of Dune


In [4]:
# two different methods to find the price
# method 1
price1 = float(soup2.find('span', {"id": "pdp-cur-price"}).get_text().strip()[1:])
print(price1)

25.99


In [5]:
# method 2
price2 = soup2.find(id="pdp-cur-price").get_text().strip()
print(price2[1:])

25.99


In [6]:
# when I automate the csv export, I want the date it was exported to be included in the report
today = datetime.date.today()

In [7]:
header = ['Book Title', 'Price', 'Date']
data = [title, price1, today]

with open('DuneWebScraper.csv', 'w', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerow(data)

In [8]:
import pandas as pd

df = pd.read_csv(r"\DuneWebScraper.csv")

print(df)

                                          Book Title  Price        Date
0  Frank Herbert's Dune Saga 3-Book Boxed Set: Du...  25.99  2024-04-01


In [9]:
# append the data

with open('DuneWebScraper.csv', 'a+', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(data)

In [10]:
# sending an email to myself when the book is selling for less than $15

def send_mail():
    server = smtplib.SMTP_SSL('smtp.gmail.com',465)
    server.ehlo()
    server.ehlo()
    server.login('<my email>','<my password>')
    
    subject = "Dune 3 Book Set is now less than $15!"
    body = "Get the Dune 3-Book Set you've been waiting for here: https://www.barnesandnoble.com/w/frank-herberts-dune-saga-3-book-boxed-set-frank-herbert/1136810577?ean=9780593201893"
    
    msg = f"Subject: {subject}\n\n{body}"
    
    server.sendmail(
        '<my email>',
        msg
     
    )

In [11]:
# function to run for automating

def check_price():
    URL = 'https://www.barnesandnoble.com/w/frank-herberts-dune-saga-3-book-boxed-set-frank-herbert/1136810577?ean=9780593201893'

    headers = {"from my computer"}

    page = requests.get(URL, headers=headers) # connecting our computer to the URL

    soup1 = BeautifulSoup(page.content, "html.parser")

    soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

    title = soup2.find_all(itemprop="name")[-1].get_text().strip()

    price1 = float(soup2.find('span', {"id": "pdp-cur-price"}).get_text().strip()[1:])
    
    import datetime
    
    today = datetime.date.today()
    
    import csv

    header = ['Book Title', 'Price', 'Date']
    data = [title, price1, today]

    with open('DuneWebScraper.csv', 'a+', newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(data)

    # send an email if the price falls below $15
    
    if(price1 < 15):
        send_mail()


In [ ]:
while(True):
    check_price()
    time.sleep(604800) # runs every week (every 604,800 seconds)